# 文档切分方法

**一、基于规则的切分方法**

根据预定义的规则和标准进行文本切分，往往缺乏灵活性和对复杂语义的深入理解，如：

* **基于字符分块** : 据固定字符数目以及特定的字符进行切分。

* **固定大小分块**：指定每个块的固定令牌数，通常会有一些重叠，以保持语义连贯性。

* **基于token的分块**：根据固定的token数进行切分，每个令牌代表一个词或语素，通常使用与目标语言模型相同的分词器。

* **内容感知分块**：使用 NLTK ，spaCy 等这工具来实现基于内容的切分，比如利用句子分割、识别段落、标题和标点符号。

* **根据规则递归分块**：递归分块首先尝试按照一定的标准（如段落或标题）分割文本，如果分割后的文本块仍然过大，就会在这些块上重复进行分割过程，直到所有块的大小都符合要求。这种方法适用于需要将长文本细分为较小片段的场景，同时尽量保持每个块的独立性和完整性。
针对特定数据的分块：这些方法尊重内容的结构和格式元素，确保语义连贯性。例如，Markdown文本可以根据标题、列表和代码块等元素进行分块，而LaTeX文本可以根据章节、小节和公式等逻辑单元进行分块。

**二、基于语义聚类的切分方法**

基于嵌入的语义分块本质上是通过一个滑动窗口（combined_sentence ）来计算相似度。那些相邻并且满足阈值的句子会被归为一个分块。如LlamaIndex中的SemanticSplitterNodeParser方法，其主要步骤如下：

1. **文本嵌入**：首先，文本被输入到一个嵌入模型中（如OpenAI的Embedding Model），该模型将文本
中的每个句子或段落转换成高维空间中的向量。这些向量代表了文本的语义特征。
2. **语义分析**：利用这些嵌入向量，可以通过计算向量之间的相似度来评估句子或段落之间的语义关系。如通过余弦相似度等度量，来确定哪些文本部分在内容上是相似的。
3. **分块决策**：基于预设的相似度阈值或其他标准，这些语义相近的文本段落被分组为一个块（chunk）。这个过程通常涉及到确定“断点”，即在哪里开始新的分块，以确保每个分块在语义上尽可能的独立和完整。

**三、基于机器学习模型的方法**

利用自然语言模型，如BERT和其他Transformer模型，这些方法通过学习文本中的语言模式来预测最合适的分块点。这些模型通常被训练来识别文本中的结构和语义断点，能够自动适应各种语言和文本类型。

* **Naive BERT**

使用BERT模型的下一句预测（NSP）功能来判断两个句子之间是否存在直接的连续关系。这种方法通过分析相邻句子的语义关系来确定分块点。

* **Cross Segment Attention[6]**

采用跨片段的注意力机制来分析文本，如通过BERT和双向LSTM结合的方式，来更细致地理解和划分文本。这种方法不仅考虑单个句子，还考虑其周围的上下文，以确定分割点。
* **SeqModel[7]**

在[6]的基础上提出了进一步的改进，即 SeqModel。SeqModel 利用 BERT 同时编码多个句子，在进行句子向量计算之前，建模了更长上下文内的依赖关系。然后，它预测每个句子之后是否会发生文本分割。此外，这个模型采用了自适应性滑动窗口方法来提高推理速度，同时不牺牲准确性。

**四、基于代理的切分方法**

通过将文本分解为独立的命题或信息单元，并根据这些单元的语义相关性进行聚类和组织。通常依赖于先进的自然语言处理技术来提取和评估文本中的关键信息，然后基于这些信息创建结构化的数据块。其步骤如下：

1.理解文本： 首先，大型语言模型（LLM）需要理解整个文本。就像一个人在阅读文章时，理解每一个段落、句子和词汇的意义一样。

2.生成命题： 然后，模型将文本分解为命题。命题是文本中的独立观点或信息片段，每个命题都包含一个完整的思想或陈述。例如，如果原文是一篇关于历史事件的文章，一个命题可能是“XX年发生了YY事件”。

3.命题评估： 接下来，模型（代理）会评估每个命题的相关性和上下文。它会考虑这个命题与文本中其他内容的关联程度。比如模型可能会问：“这个命题是否与我已经划分出的其他信息块有逻辑上的联系？”

4.创建或分配块： 基于评估结果，模型决定将命题放入现有的信息块中，还是为其创建一个新的块。如果一个命题与现有块紧密相关，它就会被添加到那个块中。如果它是一个全新的观点或信息片段，就会创建一个新的块。

5.迭代和优化： 模型会不断评估和调整，直到文本被有效地分割成一系列有意义的、相互关联的信息块。

在使用（RAG）处理长文本数据时，合理的文本切割策略是提高模型性能和效率的关键。


# 1.基于规则的切分方法

In [13]:
##将一个长文本按指定的大小（chunk_size）划分成若干个较小的段落（chunk），并将这些段落存储到一个列表（chunks）中
text = "就像交响乐需要休止符，人生存在「意义真空期」是正常的。日本茶道中的「間」（ma）哲学提醒我们：空白本身承载着张力，那些暂时迷茫的阶段，可能是新意义诞生的前奏。不要试图用单一框架解释整个人生（如「为家庭奉献」或「改变世界」）。健康的意义系统应是**「多孔结构」**：既有支撑性主干（核心价值观），又有渗透性的微意义（晨跑时看见的朝霞）。"
chunks = []
chunk_size = 128

for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
chunks


['就像交响乐需要休止符，人生存在「意义真空期」是正常的。日本茶道中的「間」（ma）哲学提醒我们：空白本身承载着张力，那些暂时迷茫的阶段，可能是新意义诞生的前奏。不要试图用单一框架解释整个人生（如「为家庭奉献」或「改变世界」）。健康的意义系统应是**「多孔结构',
 '」**：既有支撑性主干（核心价值观），又有渗透性的微意义（晨跑时看见的朝霞）。']

In [14]:
#使用 langchain 库中的 CharacterTextSplitter 类，将一个长文本按指定的大小（chunk_size）划分成若干个较小的段落（chunk），并创建包含这些段落的文档对象
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', strip_whitespace=False)
text_splitter.create_documents([text])


[Document(metadata={}, page_content='就像交响乐需要休止符，人生存在「意义真空期」是正常的。日本茶道中的「間'),
 Document(metadata={}, page_content='」（ma）哲学提醒我们：空白本身承载着张力，那些暂时迷茫的阶段，可能是'),
 Document(metadata={}, page_content='新意义诞生的前奏。不要试图用单一框架解释整个人生（如「为家庭奉献」或「'),
 Document(metadata={}, page_content='改变世界」）。健康的意义系统应是**「多孔结构」**：既有支撑性主干（'),
 Document(metadata={}, page_content='核心价值观），又有渗透性的微意义（晨跑时看见的朝霞）。')]

In [16]:
#使用 langchain 库中的 PythonCodeTextSplitter 类，将一段 Python 代码按照指定的大小（chunk_size）划分成若干个较小的段落（chunks），并创建包含这些段落的文档对象
python_text = """
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

for i in range(10):
    print (i)
"""

from langchain.text_splitter import PythonCodeTextSplitter
python_splitter = PythonCodeTextSplitter(chunk_size=100, chunk_overlap=0)
python_splitter.create_documents([python_text])


[Document(metadata={}, page_content='class Person:\n  def __init__(self, name, age):\n    self.name = name\n    self.age = age'),
 Document(metadata={}, page_content='p1 = Person("John", 36)\n\nfor i in range(10):\n    print (i)')]

In [10]:

markdown_text = """
**LangChain-Chatchat**
基于ChatGLM等大语言模型与Langchain等应用框架实现，开源、可离线部署的检索增强生成(RAG)大模型知识库项目。

## 目录
* 介绍
* 解决的痛点
* 快速上手
* 1. 环境配置
* 2. 模型下载
* 3. 初始化知识库和配置文件
* 4. 一键启动
* 5. 启动界面示例
* 联系我们

## 介绍
一种利用langchain思想实现的基于本地知识库的问答应用，目标期望建立一套对中文场景与开源模型支持友好、可离线运行的知识库问答解决方案。
一行命令运行 Docker ：
```shell
docker run -d --gpus all -p 80:8501 registry.cn-beijing.aliyuncs.com/chatchat/chatchat:0.2.7
"""

In [11]:
#使用 langchain 库中的 MarkdownTextSplitter 类，将一段 Markdown 文本按照指定的大小（chunk_size）划分成若干个较小的段落（chunks），并创建包含这些段落的文档对象
from langchain.text_splitter import MarkdownTextSplitter
splitter = MarkdownTextSplitter(chunk_size = 40, chunk_overlap=0)
splitter.create_documents([markdown_text])


[Document(metadata={}, page_content='**LangChain-Chatchat**'),
 Document(metadata={}, page_content='基于ChatGLM等大语言模型与Langchain等应用框架实现，开源、可离线'),
 Document(metadata={}, page_content='部署的检索增强生成(RAG)大模型知识库项目。'),
 Document(metadata={}, page_content='## 目录\n* 介绍\n* 解决的痛点\n* 快速上手\n* 1. 环境配置'),
 Document(metadata={}, page_content='* 2. 模型下载\n* 3. 初始化知识库和配置文件\n* 4. 一键启动'),
 Document(metadata={}, page_content='* 5. 启动界面示例\n* 联系我们'),
 Document(metadata={}, page_content='## 介绍'),
 Document(metadata={}, page_content='一种利用langchain思想实现的基于本地知识库的问答应用，目标期望建立一套'),
 Document(metadata={}, page_content='对中文场景与开源模型支持友好、可离线运行的知识库问答解决方案。'),
 Document(metadata={}, page_content='一行命令运行 Docker ：\n```shell'),
 Document(metadata={}, page_content='docker run -d --gpus all -p 80:8501'),
 Document(metadata={}, page_content='registry.cn-beijing.aliyuncs.com/chatch'),
 Document(metadata={}, page_content='at/chatchat:0.2.7')]

In [23]:
#文本将首先根据 ## 标题进行切分，然后将每个段落的长度限制在 chunk_size（40 个字符）以内
from langchain.text_splitter import MarkdownTextSplitter

class CustomMarkdownTextSplitter(MarkdownTextSplitter):
    def __init__(self, chunk_size, chunk_overlap, **kwargs):
        super().__init__(chunk_size=chunk_size, chunk_overlap=chunk_overlap, **kwargs)
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_text(self, text):
        """自定义方法，根据 `##` 标题进行切分"""
        chunks = []
        current_chunk = []

        lines = text.split('\n')
        for line in lines:
            if line.startswith('##') and current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = [line]
            else:
                current_chunk.append(line)

        if current_chunk:
            chunks.append('\n'.join(current_chunk))

        return chunks

    def create_documents(self, texts):
        """重载 create_documents 方法"""
        documents = []
        for text in texts:
            chunks = self.split_text(text)
            for chunk in chunks:
                if len(chunk) > self.chunk_size:
                    # 如果某个块超过 chunk_size，再进行二次切分
                    sub_chunks = super().split_text(chunk)
                    documents.extend(sub_chunks)
                else:
                    documents.append(chunk)
        return documents

# 创建 CustomMarkdownTextSplitter 对象
splitter = CustomMarkdownTextSplitter(
    chunk_size=100,  # 每个段落的最大字符数
    chunk_overlap=0 # 段落之间的重叠字符数
)
# 使用 splitter 对象的 create_documents 方法将 Markdown 文本划分成多个段落
documents = splitter.create_documents([markdown_text])

# 输出划分后的文档对象
for doc in documents:
    print(doc)
    print("="*40)



**LangChain-Chatchat** 
基于ChatGLM等大语言模型与Langchain等应用框架实现，开源、可离线部署的检索增强生成(RAG)大模型知识库项目。

## 目录
* 介绍
* 解决的痛点
* 快速上手
* 1. 环境配置
* 2. 模型下载
* 3. 初始化知识库和配置文件
* 4. 一键启动
* 5. 启动界面示例
* 联系我们

## 介绍
一种利用langchain思想实现的基于本地知识库的问答应用，目标期望建立一套对中文场景与开源模型支持友好、可离线运行的知识库问答解决方案。
一行命令运行 Docker ：
```shell
docker run -d --gpus all -p 80:8501 registry.cn-beijing.aliyuncs.com/chatchat/chatchat:0.2.7


# 2.基于语义聚类的切分方法

In [24]:
import numpy as np
from sentence_transformers import SentenceTransformer

def cosine_similarity(a, b):
    """计算余弦相似度"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# 初始化嵌入模型（使用小型预训练模型）
model = SentenceTransformer('all-MiniLM-L6-v2')

# 示例文本（包含5个语义相关的句子）
text = "我爱编程。编程让我快乐。今天天气很好。我打算去公园散步。散步对健康有益。"

# 简单分句处理（实际建议使用nltk的sent_tokenize）
sentences = [s.strip() for s in text.split('。') if s.strip()]

# 生成句子嵌入（768维向量）
embeddings = model.encode(sentences)

# 初始化独立块（每个句子为一个块）
chunks = [{'text': sent, 'embedding': emb} for sent, emb in zip(sentences, embeddings)]

# 设置相似度阈值（可根据任务调整）
SIMILARITY_THRESHOLD = 0.7

# 动态合并算法
changed = True
while changed:
    changed = False
    i = 0
    while i < len(chunks)-1:
        current = chunks[i]
        next_chunk = chunks[i+1]

        # 计算相邻块的语义相似度
        sim = cosine_similarity(current['embedding'], next_chunk['embedding'])

        if sim >= SIMILARITY_THRESHOLD:
            # 合并文本并生成新嵌入
            merged_text = f"{current['text']}。{next_chunk['text']}"
            merged_embedding = model.encode([merged_text])[0]

            # 更新块列表
            chunks[i] = {'text': merged_text, 'embedding': merged_embedding}
            del chunks[i+1]
            changed = True
        else:
            i += 1

# 输出最终分块结果
print("语义分块结果：")
for idx, chunk in enumerate(chunks, 1):
    print(f"[块{idx}] {chunk['text']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

语义分块结果：
[块1] 我爱编程。编程让我快乐
[块2] 今天天气很好
[块3] 我打算去公园散步
[块4] 散步对健康有益
